# Changing the input current when solving PyBaMM models

This notebook shows you how to change the input current when solving PyBaMM models. It also explains how to load in current data from a file, and how to add a user-defined current function. For more examples of different drive cycles see [here](https://github.com/pybamm-team/PyBaMM/tree/master/results/drive_cycles).

### Table of Contents
1. [Constant current](#constant)
1. [Loading in current data](#data)
1. [Adding your own current function](#function)

## Constant current  <a name="constant"></a>

In this notebook we will use the SPM as the example model, and change the input current from the default option. If you are not familiar with running a model in PyBaMM, please see [this](./models/SPM.ipynb) notebook for more details.

In PyBaMM, the current function is set using the parameter "Current function". This can be a scalar, but only accepts values 0 and 1. By default this is set to be a constant current by setting it to '1'. The size of a constant current input is changed by changing the parameter "Typical current [A]". Below we load the SPM with the default parameters, and then change the the typical current 16A. We then explicitly set the current function to be a constant current.

In [2]:
import pybamm
import numpy as np
import os
os.chdir(pybamm.__path__[0]+'/..')

# create the model
model = pybamm.lithium_ion.SPM()

# set the default model geometry
geometry = model.default_geometry

# set the default model parameters
param = model.default_parameter_values

# change the typical current and set a constant discharge using the typical current value
param["Typical current [A]"] = 16
param["Current function"] = 1


The one case where we would change "Current function" instead of "Typical current [A]" is for a simulation at zero-current, since "Typical current [A]" cannot be zero (as it is used for non-dimensionalisation). In this case, the current function should be set to zero instead:

In [3]:
param["Current function"] = 0

The model may then be processed and solved in the usual way.

In [2]:
# set the parameters for the model and the geometry
param.process_model(model)
param.process_geometry(geometry)

# mesh the domains
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise the model equations
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Solve the model at the given time points
solver = model.default_solver
n = 100
t_eval = np.linspace(0, 0.2, n)
solution = solver.solve(model, t_eval)

# plot
quick_plot = pybamm.QuickPlot(model, mesh, solution)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=0.05,value=0));

ShapeError: Cannot find shape (original error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all())

## Loading in current data <a name="data"></a>

Data can be loaded in from a csv file by specifying the path to that file and using the prefix "[current data]".

In [ ]:
param["Current function"] = "[current data]US06"

As an example, we show how to solve the SPM using the US06 drive cycle

In [ ]:
model = pybamm.lithium_ion.SPM()

# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param = model.default_parameter_values
param["Current function"] = "[current data]US06"
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# simulate US06 drive cycle (duration 600 seconds)
tau = param.process_symbol(
    pybamm.standard_parameters_lithium_ion.tau_discharge
).evaluate(0)
t_eval = np.linspace(0, 600 / tau, 600)
solution = solver.solve(model, t_eval)

# plot
quick_plot = pybamm.QuickPlot(model, mesh, solution)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=0.001,value=0));

Note that some solvers try to evaluate the model equations at a very large value of `t` during the first step. This may raise a warning if the time requested by the solver is outside of the range of the data provided. However, this does not affect the solve since this large timestep is rejected by the solver, and a suitable shorter initial step is taken.

## Adding your own current function <a name="function"></a>

A user defined current function can be passed to any model by specifying either a function or a set of data points for interpolation.

For example, you may want to simulate a sinusoidal current with amplitude A and freqency omega. In order to do so you must first define the method

In [ ]:
# create user-defined function
def my_fun(A, omega):
    def current(t):
        return A * np.sin(2 * np.pi * omega * t)
    
    return current

Note that the function returns a function which takes the input time.
Then the model may be loaded and the "Current function" parameter updated to `my_fun` called with a specific value of `A` and `omega`

In [ ]:
model = pybamm.lithium_ion.SPM()

# create geometry
geometry = model.default_geometry

# load default parameter values
param = model.default_parameter_values

# set user defined current function
A = pybamm.electrical_parameters.I_typ
omega = 0.1
param["Current function"] = my_fun(A,omega)

# process model and geometry
param.process_model(model)
param.process_geometry(geometry)

Note that when `my_fun` is evaluated with `A` and `omega`, this creates a new function `current(t)` which can then be used in the expression tree. The model may then be solved in the usual way

In [ ]:
# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# Example: simulate for 30 seconds
simulation_time = 30  # end time in seconds
tau = param.process_symbol(pybamm.standard_parameters_lithium_ion.tau_discharge).evaluate(0)
npts = 50 * simulation_time * omega  # need enough timesteps to resolve output
t_eval = np.linspace(0, simulation_time / tau, npts)
solution = model.default_solver.solve(model, t_eval)
label = ["Frequency: {} Hz".format(omega)]

# plot current and voltage
output_variables = ["Current [A]", "Terminal voltage [V]"]
quick_plot = pybamm.QuickPlot(model, mesh, solution, output_variables, label)

import ipywidgets as widgets
widgets.interact(quick_plot.plot, t=widgets.FloatSlider(min=0,max=solution.t[-1],step=solution.t[-1]/20,value=0));